# Dense model

> Fully connected neural network for action distribution.

In [ ]:
#| default_exp models.base.dense

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export 
import numpy as np
import torch
import torch.distributions as td
import torch.nn as nn
from fastcore.utils import *
from fastcore.all import *

In [ ]:
#| export
import numpy as np
import torch.nn as nn
import torch.distributions as td

class DenseModel(nn.Module):
    def __init__(
            self, 
            output_shape,
            input_size, 
            info,
        ):
        """
        :param output_shape: tuple containing shape of expected output
        :param input_size: size of input features
        :param info: dict containing num of hidden layers, size of hidden layers, activation function, output distribution etc.
        """
        super().__init__()
        self._output_shape = output_shape
        self._input_size = input_size
        self._layers = info['layers']
        self._node_size = info['node_size']
        self.activation = info['activation']
        self.dist = info['dist']
        self.model = self.build_model()


In [ ]:
#| export
@patch
def build_model(self: DenseModel):
    model = [nn.Linear(self._input_size, self._node_size)]
    model += [self.activation()]
    for i in range(self._layers-1):
        model += [nn.Linear(self._node_size, self._node_size)]
        model += [self.activation()]
    model += [nn.Linear(self._node_size, int(np.prod(self._output_shape)))]
    return nn.Sequential(*model) 

In [ ]:
#| export 
@patch
def forward(self: DenseModel, input):
    dist_inputs = self.model(input)
    if self.dist == 'normal':
        return td.independent.Independent(td.Normal(dist_inputs, 1), len(self._output_shape))
    if self.dist == 'binary':
        return td.independent.Independent(td.Bernoulli(logits=dist_inputs), len(self._output_shape))
    if self.dist == None:
        return dist_inputs

    raise NotImplementedError(self._dist)

In [ ]:
x = torch.randn(4, 10)
info = {'layers': 2, 'node_size': 32, 'activation': nn.ReLU, 'dist': 'binary'}
model = DenseModel((3,), 10, info)
out = model(x)
print(out)

Independent(Bernoulli(logits: torch.Size([4, 3])), 1)


In [ ]:
out.log_prob(torch.ones(4, 3))

tensor([-1.9618, -1.9218, -1.8859, -1.9910], grad_fn=<SumBackward1>)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()